# Exploring EHR Data for Hospital Care and Outcomes

In [44]:
# Importing the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from datetime import date
from scipy.stats import chi2_contingency

## Basic Analysis and understanding of the data

In [3]:
#Loading the datasets
patients = pd.read_csv(r"D:\DSML class\Real world data assignments\Python\EHR Capstone\PATIENTS.csv")
admissions = pd.read_csv(r"D:\DSML class\Real world data assignments\Python\EHR Capstone\ADMISSIONS.csv")
cpt = pd.read_csv(r"D:\DSML class\Real world data assignments\Python\EHR Capstone\CPTEVENTS.csv")
icu = pd.read_csv(r"D:\DSML class\Real world data assignments\Python\EHR Capstone\ICUSTAYS.csv")

## Observation of the data

In [4]:
print('\nFirst five rows of the Patients table')
patients.head(5)


First five rows of the Patients table


,row_id,subject_id,gender,dob
0,9467,10006,F,2094-03-05 0:00:00
1,9472,10011,F,2090-06-05 0:00:00
2,9474,10013,F,2038-09-03 0:00:00
3,9478,10017,F,2075-09-21 0:00:00
4,9479,10019,M,2114-06-20 0:00:00


In [5]:
patients.shape

(100, 4)

In [6]:
print('\nFirst five rows of the Admissions table')
admissions.head(5)


First five rows of the Admissions table


,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,HEPATITIS B,1
2,12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,Medicare,NaN,CATHOLIC,NaN,UNKNOWN/NOT SPECIFIED,NaN,NaN,SEPSIS,1
3,12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0
4,12270,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,NaN,NaN,ALCOHOLIC HEPATITIS,1


In [7]:
admissions.shape

(129, 17)

In [8]:
print('\nFirst five rows of the CPT events table')
cpt.head(5)


First five rows of the CPT events table


,row_id,subject_id,hadm_id,costcenter,chartdate,cpt_cd,sectionheader,subsectionheader,description
0,4615,10117,105150,ICU,NaN,99254,Evaluation and management,Consultations,NaN
1,4616,10117,105150,ICU,NaN,99231,Evaluation and management,Hospital inpatient services,NaN
2,4617,10117,105150,ICU,NaN,90935,Medicine,Dialysis,NaN
3,4618,10117,105150,ICU,NaN,99231,Evaluation and management,Hospital inpatient services,NaN
4,7753,10111,174739,ICU,NaN,99253,Evaluation and management,Consultations,NaN


In [9]:
cpt.shape

(1579, 9)

In [10]:
print('\nFirst five rows of the ICU stays table')
icu.head(5)


First five rows of the ICU stays table


,row_id,subject_id,hadm_id,icustay_id,first_careunit,last_careunit,first_wardid,last_wardid,intime,outtime,los
0,12742,10006,142345,206504,MICU,MICU,52,52,2164-10-23 21:10:15,2164-10-25 12:21:07,1.6325
1,12747,10011,105331,232110,MICU,MICU,15,15,2126-08-14 22:34:00,2126-08-28 18:59:00,13.8507
2,12749,10013,165520,264446,MICU,MICU,15,15,2125-10-04 23:38:00,2125-10-07 15:13:52,2.6499
3,12754,10017,199207,204881,CCU,CCU,7,7,2149-05-29 18:52:29,2149-05-31 22:19:17,2.1436
4,12755,10019,177759,228977,MICU,MICU,15,15,2163-05-14 20:43:56,2163-05-16 3:47:04,1.2938


In [11]:
icu.shape

(136, 11)

In [12]:
#Getting the overview of the dataset structure
patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   row_id      100 non-null    int64 
 1   subject_id  100 non-null    int64 
 2   gender      100 non-null    object
 3   dob         100 non-null    object
dtypes: int64(2), object(2)
memory usage: 3.2+ KB


In [13]:
admissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129 entries, 0 to 128
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   row_id                129 non-null    int64 
 1   subject_id            129 non-null    int64 
 2   hadm_id               129 non-null    int64 
 3   admittime             129 non-null    object
 4   dischtime             129 non-null    object
 5   deathtime             40 non-null     object
 6   admission_type        129 non-null    object
 7   admission_location    129 non-null    object
 8   insurance             129 non-null    object
 9   language              81 non-null     object
 10  religion              128 non-null    object
 11  marital_status        113 non-null    object
 12  ethnicity             129 non-null    object
 13  edregtime             92 non-null     object
 14  edouttime             92 non-null     object
 15  diagnosis             129 non-null    ob

In [14]:
cpt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1579 entries, 0 to 1578
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   row_id            1579 non-null   int64 
 1   subject_id        1579 non-null   int64 
 2   hadm_id           1579 non-null   int64 
 3   costcenter        1579 non-null   object
 4   chartdate         291 non-null    object
 5   cpt_cd            1579 non-null   int64 
 6   sectionheader     1579 non-null   object
 7   subsectionheader  1579 non-null   object
 8   description       291 non-null    object
dtypes: int64(4), object(5)
memory usage: 111.1+ KB


In [16]:
icu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136 entries, 0 to 135
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   row_id          136 non-null    int64  
 1   subject_id      136 non-null    int64  
 2   hadm_id         136 non-null    int64  
 3   icustay_id      136 non-null    int64  
 4   first_careunit  136 non-null    object 
 5   last_careunit   136 non-null    object 
 6   first_wardid    136 non-null    int64  
 7   last_wardid     136 non-null    int64  
 8   intime          136 non-null    object 
 9   outtime         136 non-null    object 
 10  los             136 non-null    float64
dtypes: float64(1), int64(6), object(4)
memory usage: 11.8+ KB


In [17]:
#Checking the summary of numerical columns in patients table
patients.describe()

,row_id,subject_id
count,100.000000,100.000000
mean,20452.580000,26162.330000
std,10982.043163,16201.834705
min,9467.000000,10006.000000
25%,9526.500000,10068.500000
50%,20209.000000,25128.000000
75%,31391.250000,42276.500000
max,31872.000000,44228.000000


In [18]:
#Checking the summary of numerical columns in admissions table
admissions.describe()

,row_id,subject_id,hadm_id,hospital_expire_flag
count,129.000000,129.000000,129.000000,129.000000
mean,28036.441860,28010.410853,152343.441860,0.310078
std,14036.548988,16048.502883,27858.788248,0.464328
min,12258.000000,10006.000000,100375.000000,0.000000
25%,12339.000000,10088.000000,128293.000000,0.000000
50%,39869.000000,40310.000000,157235.000000,0.000000
75%,40463.000000,42135.000000,174739.000000,1.000000
max,41092.000000,44228.000000,199395.000000,1.000000


In [19]:
#Checking the summary of numerical columns in cpt table
cpt.describe()

,row_id,subject_id,hadm_id,cpt_cd
count,1579.000000,1579.000000,1579.000000,1579.000000
mean,254352.809373,30132.860038,157868.511083,94952.146928
std,152033.453716,15398.361505,27904.711248,13529.257441
min,4615.000000,10006.000000,100375.000000,11043.000000
25%,136183.500000,10120.000000,138132.000000,94003.000000
50%,238558.000000,40310.000000,163189.000000,99232.000000
75%,275015.500000,42135.000000,182839.000000,99253.000000
max,543399.000000,44228.000000,199395.000000,99367.000000


In [20]:
#Checking the summary of numerical columns in icu table
icu.describe()

,row_id,subject_id,hadm_id,icustay_id,first_wardid,last_wardid,los
count,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000,136.000000
mean,29370.316176,28263.485294,153259.566176,250980.470588,34.470588,34.022059,4.452457
std,14550.435239,16008.281510,28054.220280,28455.125832,18.876182,19.280951,6.196828
min,12742.000000,10006.000000,100375.000000,201006.000000,7.000000,7.000000,0.105900
25%,12826.750000,10089.750000,129028.000000,224359.250000,14.750000,14.000000,1.233525
50%,41427.000000,40307.000000,157724.000000,250055.000000,33.000000,33.000000,2.111450
75%,42073.750000,42207.000000,174896.500000,277279.250000,52.000000,52.000000,4.329050
max,42714.000000,44228.000000,199395.000000,298685.000000,57.000000,57.000000,35.406500


In [21]:
#Unique values and it's count unique of all columns
print('\nUnique values of categorical columns in patients table')
(patients.select_dtypes(include='object')).nunique()


Unique values of categorical columns in patients table


gender     2
dob       99
dtype: int64

In [22]:
print('\nUnique values of categorical columns in admissions table')
(admissions.select_dtypes(include='object')).nunique()


Unique values of categorical columns in admissions table


admittime             129
dischtime             129
deathtime              40
admission_type          3
admission_location      5
insurance               4
language                5
religion               10
marital_status          6
ethnicity               9
edregtime              92
edouttime              92
diagnosis              95
dtype: int64

In [23]:
print('\nUnique values of categorical columns in cpt table')
(cpt.select_dtypes(include='object')).nunique()


Unique values of categorical columns in cpt table


costcenter            2
chartdate           284
sectionheader         4
subsectionheader     21
description           4
dtype: int64

In [24]:
print('\nUnique values of categorical columns in icu table')
(icu.select_dtypes(include='object')).nunique()


Unique values of categorical columns in icu table


first_careunit      5
last_careunit       5
intime            136
outtime           136
dtype: int64

## Data processing

In [25]:
#Checking for missing values
patients.isna().sum()

row_id        0
subject_id    0
gender        0
dob           0
dtype: int64

In [41]:
admissions.isna().sum()

row_id                   0
subject_id               0
hadm_id                  0
admittime                0
dischtime                0
deathtime               89
admission_type           0
admission_location       0
insurance                0
language                48
religion                 1
marital_status          16
ethnicity                0
edregtime               37
edouttime               37
diagnosis                0
hospital_expire_flag     0
dtype: int64

In [48]:
imp = SimpleImputer(strategy='most_frequent')
missing_cols = ['language', 'religion', 'marital_status']
for column in missing_cols:
    admissions[column] = pd.DataFrame(imp.fit_transform(pd.DataFrame(admissions[column])))
admissions.isna().sum()

row_id                   0
subject_id               0
hadm_id                  0
admittime                0
dischtime                0
deathtime               89
admission_type           0
admission_location       0
insurance                0
language                 0
religion                 0
marital_status           0
ethnicity                0
edregtime               37
edouttime               37
diagnosis                0
hospital_expire_flag     0
dtype: int64

In [43]:
icu.isna().sum()

row_id            0
subject_id        0
hadm_id           0
icustay_id        0
first_careunit    0
last_careunit     0
first_wardid      0
last_wardid       0
intime            0
outtime           0
los               0
dtype: int64

In [42]:
cpt.isna().sum()

row_id                 0
subject_id             0
hadm_id                0
costcenter             0
chartdate           1288
cpt_cd                 0
sectionheader          0
subsectionheader       0
description         1288
dtype: int64